In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob


In [2]:
DATA_ROOT = '/home/becode/Skybase/new_data'
valid_path = '/home/becode/Skybase/valid_data/valid'

In [3]:
#categories
root=pathlib.Path(DATA_ROOT)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes

['Negative', 'Positive']

In [4]:


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [5]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=2)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=2, bias=True)
)

In [6]:

#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [7]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [8]:
images_path=glob.glob(valid_path+'/*.jpg')

In [9]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

/home/becode/Skybase/pytorch/pytenv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [10]:
pred_dict

{'7069-6.jpg': 'Negative',
 '7069-97.jpg': 'Negative',
 '7069-159.jpg': 'Negative',
 '7069-50.jpg': 'Negative',
 '7069-174.jpg': 'Negative',
 'C16.jpg': 'Positive',
 'C15.jpg': 'Positive',
 '7069-79.jpg': 'Negative',
 '7069-168.jpg': 'Negative',
 '7069-184.jpg': 'Negative',
 '7069-31.jpg': 'Negative',
 '7069-180.jpg': 'Negative',
 '7069-102.jpg': 'Negative',
 'C46.jpg': 'Positive',
 '7069-86.jpg': 'Negative',
 'C70.jpg': 'Negative',
 '7069-93.jpg': 'Negative',
 'C26.jpg': 'Positive',
 '7069-134.jpg': 'Negative',
 'C24.jpg': 'Positive',
 '7069-219.jpg': 'Negative',
 'C67.jpg': 'Positive',
 '7069-96.jpg': 'Negative',
 '7069-181.jpg': 'Negative',
 '7069-112.jpg': 'Negative',
 'C37.jpg': 'Positive',
 '7069-135.jpg': 'Negative',
 '7069-95.jpg': 'Negative',
 '7069-133.jpg': 'Negative',
 'C45.jpg': 'Positive',
 '7069-71.jpg': 'Negative',
 'C9.jpg': 'Positive',
 'C34.jpg': 'Positive',
 '7069-78.jpg': 'Negative',
 'C31.jpg': 'Positive',
 'C56.jpg': 'Positive',
 '7069-187.jpg': 'Negative',
 '706